In [19]:
import pandas as pd
import numpy as np
from slugify import slugify
pd.set_option('display.max_columns', None)

In [24]:
df = pd.read_csv("../data/raw/iowa_liquor_train_test_split_sample.csv")

In [28]:
df.columns = [slugify(col, lowercase=True, separator='_') for col in df.columns]
df.columns

Index(['unnamed_0', 'invoice_item_number', 'date', 'store_number',
       'store_name', 'address', 'city', 'zip_code', 'store_location',
       'county_number', 'county', 'category', 'category_name', 'vendor_number',
       'vendor_name', 'item_number', 'item_description', 'pack',
       'bottle_volume_ml', 'state_bottle_cost', 'state_bottle_retail',
       'bottles_sold', 'sale_dollars', 'volume_sold_liters',
       'volume_sold_gallons'],
      dtype='object')

In [29]:
df.head(1)

,unnamed_0,invoice_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,2364808,S10798700028,02/26/2013,4347,Jeff's Market / Blue Grass,102 W MAYNE,BLUE GRASS,52726,"102 W MAYNE\nBLUE GRASS 52726\n(41.509114, -90...",82.0,Scott,1011200.0,STRAIGHT BOURBON WHISKIES,461.0,Campari(skyy),22155,Wild Turkey 101 Round,12,375,$6.00,$8.99,1,$8.99,0.38,0.1


In [33]:
df = df.drop(columns=['unnamed_0', 'invoice_item_number','store_number','store_name','address','zip_code','category','vendor_number','vendor_name','item_number','pack','bottle_volume_ml','state_bottle_cost','bottles_sold','volume_sold_gallons'])

In [34]:
df.head()

,date,city,store_location,county_number,county,category_name,item_description,state_bottle_retail,sale_dollars,volume_sold_liters
0,02/26/2013,BLUE GRASS,"102 W MAYNE\nBLUE GRASS 52726\n(41.509114, -90...",82.0,Scott,STRAIGHT BOURBON WHISKIES,Wild Turkey 101 Round,$8.99,$8.99,0.38
1,08/22/2017,Indianola,1500 North Jefferson\nIndianola 50125\n,91.0,WARREN,Canadian Whiskies,Crown Royal Canadian Whisky,$47.99,$287.94,1.75
2,08/28/2013,STORM LAKE,"1250 N LAKE ST\nSTORM LAKE 50588\n(42.653594, ...",11.0,Buena Vista,DISTILLED SPIRITS SPECIALTY,Firefly Strawberry Moonshine,$14.82,$29.64,1.50
3,03/26/2013,MANNING,"316 MAIN ST\nMANNING 51455\n(41.908573, -95.06...",14.0,Carroll,IMPORTED SCHNAPPS,Rumple Minze Peppermint Schnapps Liqueur,$22.72,$22.72,1.00
4,08/17/2017,Des Moines,"218, 6th Ave #101\nDes Moines 50309\n(41.58568...",77.0,POLK,Mixto Tequila,Jose Cuervo Especial Reposado Tequila Mini,$17.25,$17.25,0.50


In [46]:
df.shape

(2518216, 10)

In [40]:
df[df['date'].isna()].shape

(0, 10)

In [39]:
df[df['city'].isna()].shape

(467, 10)

In [41]:
df[df['store_location'].isna()].shape

(467, 10)

In [42]:
df[df['county_number'].isna()].shape

(15650, 10)

In [43]:
df[df['county'].isna()].shape

(15650, 10)

In [44]:
df[df['category_name'].isna()].shape

(3335, 10)